In [ ]:
pip install split-folders

In [ ]:
#CREATE DIRECTORIES
import os
import shutil


working_dir = './training'
if(os.path.isdir(working_dir)):
    shutil.rmtree(working_dir)
os.mkdir(working_dir)
os.mkdir(os.path.join(working_dir, 'NO PERSON'))
os.mkdir(os.path.join(working_dir, 'ALL THE PEOPLE'))
os.mkdir(os.path.join(working_dir, 'SOMEONE'))

In [ ]:
#IMPORT STUFF + TEST JSON
import json
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from shutil import copy
from PIL import Image

with open('../input/artificial-neural-networks-and-deep-learning-2020/MaskDataset/train_gt.json') as f:
    labels = json.load(f) # crea dizionario in python
    print(labels['14985.jpg'])

In [ ]:
dataset_dir = '/kaggle/input/artificial-neural-networks-and-deep-learning-2020/MaskDataset'
training_dir = os.path.join(dataset_dir, 'training')
training_dir

dataset_dir = '/kaggle/input/artificial-neural-networks-and-deep-learning-2020/MaskDataset'
test_dir = os.path.join(dataset_dir, 'test')
test_dir

In [ ]:
#COPY IMAGES IN RESPECTIVE FOLDERS
dataset_dir = '/kaggle/input/artificial-neural-networks-and-deep-learning-2020/MaskDataset'
training_dir = os.path.join(dataset_dir, 'training')

for dirname, _, filenames in os.walk(training_dir):
    for filename in filenames:
        if labels[filename] == 0:
            copy(os.path.join(dirname, filename), os.path.join(working_dir, 'NO PERSON') )
        elif labels[filename] == 1:
            copy(os.path.join(dirname, filename), os.path.join(working_dir, 'ALL THE PEOPLE') )
        elif labels[filename] == 2:
            copy(os.path.join(dirname, filename), os.path.join(working_dir, 'SOMEONE') )

In [ ]:
import numpy as np 
#import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import os
import tensorflow as tf
import splitfolders

SEED = 1234
splitfolders.ratio(working_dir, output="output", seed=SEED, ratio=(0.85, 0.15), group_prefix=None)
tf.random.set_seed(SEED)  

# Get current working directory
cwd = os.getcwd()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

apply_data_augmentation = True

if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(rotation_range=20,
                                        #width_shift_range=0.2,
                                        height_shift_range=0.2,
                                        horizontal_flip=True,
                                        #vertical_flip=True,
                                        fill_mode='constant',
                                        cval=0,
                                        rescale=1./255)
    valid_data_gen = ImageDataGenerator(rescale=1./255)
else:
    valid_data_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Create generators to read images from dataset directory
# -------------------------------------------------------

# Batch size
bs = 10 

# img shape
img_h = 256
img_w = 256

num_classes=3

decide_class_indices = True
if decide_class_indices:
    classes = ['NO PERSON',        # 0
               'ALL THE PEOPLE',   # 1
               'SOMEONE']          # 2
else:
    classes=None

# Training
train_gen = train_data_gen.flow_from_directory("./output/train",
                                               batch_size=bs,
                                               classes=classes,
                                               class_mode='categorical', 
                                               shuffle=True,
                                               target_size=(img_h,img_w),
                                               seed=SEED)

# Validation
valid_gen = valid_data_gen.flow_from_directory("./output/val",
                                               batch_size=bs, 
                                               classes=classes,
                                               class_mode='categorical',
                                               shuffle=True,
                                                target_size=(img_h,img_w),
                                               seed=SEED)


In [ ]:
train_gen.class_indices

In [ ]:
# Create Dataset objects
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))
train_dataset = train_dataset.repeat()

valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))
valid_dataset = valid_dataset.repeat()

In [ ]:
start_f = 12 
depth = 7

model = tf.keras.Sequential()

# Features extraction
for i in range(depth):

    if i == 0:
        input_shape = [img_h, img_w, 3]
    else:
        input_shape=[None]

    # Conv block: Conv2D -> Activation -> Pooling
    model.add(tf.keras.layers.Conv2D(filters=start_f, 
                                     kernel_size=(3, 3), # invece di 3x3
                                     strides=(1, 1),
                                     padding='same',
                                     input_shape=input_shape))
    model.add(tf.keras.layers.LeakyReLU()) # provo leaky relu e non relu
    #if i!=(depth-1):
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))

    start_f *= 2

#model.add(tf.keras.layers.GlobalAveragePooling2D())

# Classifier
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=512, activation=tf.keras.activations.relu, kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(units=512, activation=tf.keras.activations.relu, kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(tf.keras.layers.Dropout(0.3))
#model.add(tf.keras.layers.Dense(units=512, activation=tf.keras.activations.relu, kernel_regularizer=tf.keras.regularizers.L2(0.001)))
#model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))
model.summary()

In [ ]:
# Optimization params
# -------------------

# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Validation metrics
# ------------------

metrics = ['accuracy']
# ------------------

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
from datetime import datetime


cwd = os.getcwd()

exps_dir = os.path.join('./', 'classification_experiments')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name = 'CNN'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []

# Model checkpoint
# ----------------
ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'), 
                                                   save_weights_only=True)  # False to save the model directly
callbacks.append(ckpt_callback)

# Visualize Learning on Tensorboard
# ---------------------------------
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
    
# By default shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=1)  # if 1 shows weights histograms
callbacks.append(tb_callback)

# Early Stopping
# --------------
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7)
    callbacks.append(es_callback)

In [ ]:
tf.device('/device:GPU:0')
history = model.fit(x=train_dataset,
          epochs=100,  
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_gen), 
          callbacks=callbacks)

history.history

In [ ]:
#model.save_weights("./weights_saved/cp_23.ckpt")

In [ ]:
#model.load_weights('../classification_experiments/CNN_Nov22_08-59-57/ckpts/cp_37.ckpt')

In [ ]:
import os
from datetime import datetime

def create_csv(results, results_dir='/kaggle/working'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')


In [ ]:
from PIL import Image
from PIL import ImageOps

test_dir = os.path.join(dataset_dir, 'test')
image_filenames = next(os.walk(test_dir))[2]

results = {}
for image_name in image_filenames:

    img = Image.open(os.path.join(test_dir, image_name)).convert('RGB') 
   
    maxsize = (img_h, img_w)
    img = ImageOps.fit(img, maxsize, Image.ANTIALIAS)
    img_array = np.array(img)
    img_array = np.expand_dims(img_array, 0)
    img_array = img_array/255
   
    prediction = np.argmax(model.predict(img_array))   # predicted class
  
    results[image_name] = prediction

create_csv(results)